# TODO

* [ x ] Modelo Checkpoint
* [ ] Mostrar previsão de validação durante o treinamento
* [ ] Criar os gráficos val_acc e val_loss
* [ ] Mostrar o gráfico de acurácia durante o treinamento

# 1. import the necessery libraries


In [ ]:
import tensorflow as tf
import numpy as np
import pickle

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers


In [ ]:
# DateTimeStamp para taggear arquivos e gerar "versionamento" para evitar sobrescritas

from datetime import datetime
fmtedDateTime = datetime.now().strftime('%Y%m%d%H%M%S')
print(fmtedDateTime)

# 2. load and split the data 

In [ ]:
import os 
import numpy as np 
import pandas as pd 
import imageio 
import matplotlib.pyplot as plt 
%matplotlib inline 

path =''
image_path = os.path.join(path,'../input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images/')
mask_path = os.path.join(path,'../input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/')
image_list = sorted(os.listdir(image_path))
mask_list = sorted(os.listdir(mask_path))
image_list = [image_path+i for i in image_list]
mask_list = [mask_path+i for i in mask_list]
image_list = sorted(image_list)
mask_list = sorted(mask_list)
train_image_list = image_list[:390]
train_mask_list = mask_list[:390]

validation_image_list = image_list[390:400]
validation_mask_list = mask_list[390:400]

print("number of train images is : {} ".format(len(train_image_list)))
print("number of train masks is : {} ".format(len(train_mask_list)))

print("number of train images is : {} ".format(len(validation_image_list)))
print("number of train masks is : {} ".format(len(validation_mask_list)))

print(image_list[0])
print(mask_list[0])

In [ ]:
print("Number of samples:", len(image_list))

for input_path, target_path in zip(image_list[:10], mask_list[:10]):
    print(input_path, "|", target_path)

# 3. explore some images :

In [ ]:
n = 10 # you can chose any index 
img  = imageio.imread(train_image_list[n])
print(img.shape)
mask = imageio.imread(train_mask_list[n])
print(mask.shape)

# now let's plot 
fig ,arr  = plt.subplots(1,2,figsize=(15,10))
arr[0].imshow(img)
arr[0].set_title('Original Image')
arr[1].imshow(mask)
arr[1].set_title('Mask')

In [ ]:
train_images = tf.constant(train_image_list)
train_masks = tf.constant(train_mask_list)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_masks))
for image,mask in train_dataset.take(1) : 
    print(image)
    print(mask)
    
validation_images = tf.constant(validation_image_list)
validation_masks = tf.constant(validation_mask_list)

validation_dataset = tf.data.Dataset.from_tensor_slices((validation_images,validation_masks))
for image,mask in validation_dataset.take(1) : 
    print(image)
    print(mask)    

# 4. preprocessing our data

In [ ]:
def process_path(image_path,mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.image.convert_image_dtype(img,tf.float32) #this do the same as dividing by 255 to set the values between 0 and 1 (normalization)
    
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask,channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img , mask

def preprocess(image,mask) : 
    input_image = tf.image.resize(image,(96,128),method='nearest')
    input_mask = tf.image.resize(mask,(96,128),method='nearest')
    
    return input_image , input_mask

train_image_ds = train_dataset.map(process_path) # apply the preprocces_path function to our train_dataset
print(train_image_ds)
train_processed_image_ds = train_image_ds.map(preprocess) # apply the preprocess function to our train_dataset

validation_image_ds = validation_dataset.map(process_path) # apply the preprocces_path function to our validation_dataset
print(validation_image_ds)
validation_processed_image_ds = validation_image_ds.map(preprocess) 


# 5.  Define The Conv Block For The Contracting Path


In [ ]:
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):
    
    conv = Conv2D(n_filters, 
                  kernel_size = 3,     
                  activation='relu',
                  padding='same',
                  kernel_initializer=tf.keras.initializers.HeNormal())(inputs)

    conv = Conv2D(n_filters, 
                  kernel_size = 3, 
                  activation='relu',
                  padding='same',
                  kernel_initializer=tf.keras.initializers.HeNormal())(conv)

    if dropout_prob > 0:
        conv = Dropout(dropout_prob)(conv)
        
    if max_pooling:
        next_layer = MaxPooling2D(pool_size=(2,2))(conv)

    else:
        next_layer = conv
        
    skip_connection = conv
    
    return next_layer, skip_connection

# 6. Define the upsampling block for the expanding path

In [ ]:
def upsampling_block(expansive_input, contractive_input, n_filters=32):
    
    up = Conv2DTranspose(
                 n_filters,  
                 kernel_size = 3,
                 strides=(2,2),
                 padding='same')(expansive_input)
    
    merge = concatenate([up, contractive_input], axis=3)
    
    conv = Conv2D(n_filters,  
                 kernel_size = 3,   
                 activation='relu',
                 padding='same',
                 kernel_initializer=tf.keras.initializers.HeNormal())(merge)
    
    conv = Conv2D(n_filters,  
                 kernel_size = 3,  
                 activation='relu',
                 padding='same',
                 kernel_initializer=tf.keras.initializers.HeNormal())(conv)
    
    return conv

# 7. Finally! ,  we will Define the unet model 
## which composes of a set of conv blocks and upsampling blocks

In [ ]:
def build_unet_model(input_size=(96, 128, 3), n_filters=32, n_classes=23):
    
    inputs = Input(input_size)
    
    # contracting path
    cblock1 = conv_block(inputs, n_filters, dropout_prob=0.4)
    cblock2 = conv_block(cblock1[0], 2*n_filters, dropout_prob=0.4)
    cblock3 = conv_block(cblock2[0], 4*n_filters, dropout_prob=0.4)
    cblock4 = conv_block(cblock3[0], 8*n_filters, dropout_prob=0.45) 
    cblock5 = conv_block(cblock4[0],16*n_filters, dropout_prob=0.4, max_pooling=None)     
    
    # expanding path
    ublock6 = upsampling_block(cblock5[0], cblock4[1],  8 * n_filters)
    ublock7 = upsampling_block(ublock6, cblock3[1],  n_filters*4)
    ublock8 = upsampling_block(ublock7,cblock2[1] , n_filters*2)
    ublock9 = upsampling_block(ublock8,cblock1[1],  n_filters)

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)
    
    conv10 = Conv2D(n_classes, kernel_size=1, padding='same')(conv9)  
    model = tf.keras.Model(inputs=inputs, outputs=conv10, name=f"{fmtedDateTime}_U-Net")

    return model

In [ ]:
img_height = 96
img_width = 128
num_channels = 3

unet = build_unet_model((img_height, img_width, num_channels))

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img

def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

def show_predictions(dataset=None, num=1):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = unet.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])

    else:
        display([sample_image, sample_mask,
             create_mask(unet.predict(sample_image[tf.newaxis, ...]))])

# 8. Our model is ready !!

In [ ]:
unet.summary()

In [ ]:
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Incluindo alguns Callbacks e provas visuais de acurácia
Fontes de estudo:

**Callbacks e 
ModelCheckPoint**

* Aulas BI_Master
* https://machinelearningmastery.com/check-point-deep-learning-models-keras/
* https://pyimagesearch.com/2021/06/30/how-to-use-the-modelcheckpoint-callback-with-keras-and-tensorflow/


Criando Checkpoint

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
# Salvando apenas os pesos, logo, para se utilizar o modelo posteriormente,
# será preciso reconstruí-lo e carregar os pesos
checkpoint_filepath = f"/kaggle/working/{fmtedDateTime}_bestModelWeigths.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stop = EarlyStopping(monitor='val_accuracy', patience=20)

Criando callback de visualização parcial das inferências durante o treino

In [ ]:
def display_learning_curves(history):
    history = history
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]

    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

#     epochs_range = range(NUM_EPOCHS)

    fig = plt.figure(figsize=(30, 5))

    # summarize history for accuracy
    plt.subplot(1,2,1)
    plt.plot(acc, label="train accuracy")
    plt.plot(val_acc, label="validataion accuracy")
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
    # summarize history for loss
    plt.subplot(1,2,2)
    plt.plot(loss, label="train loss")
    plt.plot(val_loss, label="validataion loss")
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
#     plt.subplot(1,2,1)
#     plt.plot(epochs_range, acc, label="train accuracy")
#     plt.plot(epochs_range, val_acc, label="validataion accuracy")
#     plt.title("Accuracy")
#     plt.xlabel("Epoch")
#     plt.ylabel("Accuracy")
#     plt.legend(loc="lower right")

#     plt.subplot(1,2,2)
#     plt.plot(epochs_range, loss, label="train loss")
#     plt.plot(epochs_range, val_loss, label="validataion loss")
#     plt.title("Loss")
#     plt.xlabel("Epoch")
#     plt.ylabel("Loss")
#     plt.legend(loc="upper right")

    fig.tight_layout()
    plt.show()


In [ ]:
from tensorflow.keras.callbacks import Callback

class DisplayCallback(Callback):
    def __init__(self, model):
        self.model = model
        
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 10 == 0:
            print("--------- partial results ---------")
            display_learning_curves(self.model.history)
            print("------- partial predictions -------")
            show_predictions(train_dataset, 1)            

In [ ]:
import inspect

inspect.getfullargspec(unet.fit)

In [ ]:
EPOCHS = 1000
VAL_SUBSPLITS = 5
BUFFER_SIZE = 390
BATCH_SIZE = 64

# train
train_processed_image_ds.batch(BATCH_SIZE)
train_dataset = train_processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# print(train_processed_image_ds.element_spec)

# validation
validation_processed_image_ds.batch(BATCH_SIZE)
validation_dataset = validation_processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# print(validation_processed_image_ds.element_spec)
 
# callbacks_list = [model_checkpoint_callback, early_stop, DisplayCallback()]
callbacks_list = [DisplayCallback(unet), early_stop, model_checkpoint_callback]
model_history = unet.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    verbose=1)

In [ ]:
plt.plot(model_history.history["accuracy"])

In [ ]:
# summarize history for accuracy
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Salvando o Model History caso seja necessário buscar métricas no futuro

In [ ]:
def save_history(history, dest_path):
    with open(dest_path, "wb") as file_hist:
        pickle.dump(history.history, file_hist)

In [ ]:
save_history(model_history, f"/kaggle/working/{fmtedDateTime}_ModelHistory.pkl")

In [ ]:
fmtedDateTime

Carregar o Model History

In [ ]:
def load_history(history_path):
    history = None
    with open(history_path, "rb") as file_hist:
        history = pickle.load(file_hist)
        
    return history

In [ ]:
history = load_history(f"/kaggle/working/{fmtedDateTime}_ModelHistry.pkl")

In [ ]:
for idx, key in enumerate(history):
    print(idx, key)

In [ ]:
# model evaluation
print("Unet Model Evaluation: ")
unet.evaluate(validation_dataset)

In [ ]:
unet.load_weights("/kaggle/working/20230612183426_bestModelWeigths.h5")

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
print("----- training_dataset results---------")
show_predictions(train_dataset, 6)
print("----- validation_dataset results---------")
show_predictions(validation_dataset, 6)

In [ ]:
def save_predicted_mask(image, image_name):
    predicted_mask_path = "/kaggle/working/predicted_mask"
    if os.

In [ ]:
def reprocess(image,mask) : 
    input_image = tf.image.resize(image,(4000,6000),method='nearest')
    input_mask = tf.image.resize(mask,(4000,6000),method='nearest')
    
    return input_image , input_mask